In [ ]:
from MyNeuralNet import FirstOrderSolver
from MyNeuralNet import QuasiNewtonSolver
from MyNeuralNet import NeuralNet
import matplotlib.pyplot as plt
import numpy as np

x=np.array([np.linspace(0,2*np.pi,1000)])
y=x*np.sin(x)+np.cos(3*x)
# Initialize Neural Net
NN=NeuralNet([1,5,5,5,1],'tanh')


# Optimization
NN=FirstOrderSolver(x,y,'ADAM',MaxIter=400,StepSize=2e-3,BatchSize=100,Net=NN,m1=0.9,m2=0.999)
NN=QuasiNewtonSolver(x,y,NN,400)
# Visualization
y0=NN.Evaluate(x)

plt.figure
plt.plot(x[0,:],y0[0,:])
plt.plot(x[0,:],y[0,:])
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
# NetStruct = [InputSize,HiddenSize,HiddenSize,HiddenSize,OutputSize]
class TorchNN(nn.Module):
    def __init__(self, InputSize, HiddenSize, OutputSize):
        super(TorchNN, self).__init__()
        self.fc1 = nn.Linear(InputSize, HiddenSize)
        self.fc2 = nn.Linear(HiddenSize, HiddenSize)
        self.fc3 = nn.Linear(HiddenSize, HiddenSize)
        self.fc4 = nn.Linear(HiddenSize, OutputSize)
        self.activation = nn.Tanh()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

# Generate Data

x = np.linspace(0, 2*np.pi, 1000)
y = x*np.sin(x)+np.cos(3*x)
xTensor = torch.tensor(x, dtype=torch.float32).view(-1, 1)
yTensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)

# Set Up LayeyStruct
InputSize = 1
HiddenSize = 5
OutputSize = 1
model = TorchNN(InputSize, HiddenSize, OutputSize)

# Solver Set Up
optimizer = optim.Adam(model.parameters(),lr=3e-3)
Cost = nn.MSELoss()

# Parameters Set Up
batch_size = 200
MaxIter = 1000
num_batches = len(x) // batch_size

# Iterations
for i in range(MaxIter):
    tl = 0
    for j in range(num_batches):
        StartIndex = j * batch_size
        EndIndex = (j + 1) * batch_size
        sx = xTensor[StartIndex:EndIndex]
        sy = yTensor[StartIndex:EndIndex]

        optimizer.zero_grad()
        outputs = model(sx)
        loss = Cost(outputs, sy)
        loss.backward()
        optimizer.step()

        tl = tl+loss.item()

    if (i + 1) % 100 == 0:
        print(f"Iteration {i+1}, Cost: {tl:.4f}")

# Evaluate
input_value = torch.tensor([[1.0]], dtype=torch.float32)
with torch.no_grad():
    prediction = model(input_value)
predicted_value = prediction.item()



# Visualization
x_test = np.linspace(0, 2*np.pi, 1000)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).view(-1, 1)
with torch.no_grad():
    y_test_predicted = model(x_test_tensor).numpy()

MyNetX = np.array([np.linspace(0, 2*np.pi, 1000)])
MyNetY = x*np.sin(x)+np.cos(3*x)
y0=NN.Evaluate(MyNetX)
plt.plot(x, y, label="sin(x)")
plt.plot(x_test, y_test_predicted, label="NN Approximation")
# plt.plot(x_train[0,:], y0[0,:], label='MyNet')

plt.legend()
plt.xlabel("x")
plt.ylabel("y")
plt.show()